In [60]:
# This code follows is a modified version of the tutorial from ligo.skymap
from astropy.coordinates import SkyCoord

In [81]:
# Import main packages
from tilepy.include.CampaignDefinition import ObservationParameters
from tilepy.include.PointingTools import FilterGalaxies, LoadGalaxies, getdate

skymap_name = "https://gracedb.ligo.org/api/superevents/S240615dg/files/Bilby.offline0.multiorder.fits"
obsTime = getdate("2025-03-28 19:10:00")
pointingsFile = None

# Path to the input datasets (where galaxy cat should be for GW case)
datasetDir = "../../src/tilepy/dataset/"

# Path to the output folder
outDir = "./output"

# Config file for the tiling scheduling
cfgFile = "../config/FollowupParameters_LST.ini"

galcatName = "../../src/tilepy/dataset/Gladeplus.h5"

obspar = ObservationParameters()
obspar.add_parsed_args(
    skymap_name, obsTime, datasetDir, galcatName, outDir, pointingsFile
)
obspar.from_configfile(cfgFile)
obspar.distCut = 1000000  # Otherwise it will think that we shouldnt use a catalog
print(obspar)

============== Observation Parameters ======================
Observatory Name: LST
Event Name: None
obsTime: 2025-03-28 19:10:00
---------------------- Strategy ----------------------
Algorithm = 3D, Strategy = integrated,  Mangrove = False
Do Plot = True, Do Rank = True, Count Previous= True, Second Round= False, Use Grey Time= False
--------------------- Observatory ---------------------
Observatory Location: 28.761944 deg, -17.89 deg, 2200.0 m
FOV: 2.5, Duration: 20.0, Min Duration: 10.0, Min Slewing: 0.0
Max Runs: 4, Max Nights: 1
Visibility: -18, -0.5, 65, 0
Min Moon Source Separation: 30
Max Moon Source Separation: 145
Max Zenith: 70, Zenith Weighting: 0.75
--------------------- Skymap considerations ----------------
Skymap: https://gracedb.ligo.org/api/superevents/S240615dg/files/Bilby.offline0.multiorder.fits
Cuts: MinProbcut 0.02, Dist Cut: 1000000, Minimum Prob Cut for Catalogue: 0.01
Percentage MOC: 0.9
NSIDE: HR = 512, reduced = 256
--------------------- Directories and fil

In [62]:
catalog = "../../src/tilepy/dataset/Gladeplus.h5"
cat = LoadGalaxies(catalog)

Loading galaxy catalogue from ../../src/tilepy/dataset/Gladeplus.h5


In [63]:
cat

no_GLADE,RAJ2000,DEJ2000,Dist,Bmag
uint32,float64,float64,float32,float32
1078404,209.660614,28.544758,307.755,17.04
1078382,150.492096,27.61278,390.2103,17.033
1078358,171.639633,26.348652,216.03331,16.671
1078355,155.931229,26.182703,293.52972,16.539
1078350,160.374588,25.971264,356.17014,16.558
1078319,181.437912,24.259342,234.59818,16.125
1078292,162.406204,22.255919,226.93306,16.213
1078251,218.096588,19.566315,243.14256,17.153
1078114,159.47876,10.863852,433.2221,16.338


In [64]:
from astropy import units as u

coordinates = SkyCoord(
    cat["RAJ2000"] * u.deg, cat["DEJ2000"] * u.deg, cat["Dist"] * u.Mpc
)

In [66]:
from tilepy.include.MapManagement import SkyMap, create_map_reader

raw_map = create_map_reader(obspar)
skymap = SkyMap(obspar, raw_map)
nside = obspar.HRnside
prob = skymap.getMap("prob", obspar.HRnside)

if skymap.is3D:
    print("Skymap is 3D")
else:
    print("Skymap is 2D")

# correlate GW map with galaxy catalog, retrieve ordered list
cat = skymap.computeGalaxyProbability(cat)
tGals0 = FilterGalaxies(cat, obspar.minimumProbCutForCatalogue)

The filename is Bilby.offline0.multiorder.fits
The file exists, it will be re-downloaded


Skymap is 3D


In [79]:
import numpy as np

# Filter for dp_dV > 2e-15

filtered = cat[cat["dp_dV"] > 2e-15]

# Sort in descending order
sorted_filtered = filtered[np.argsort(filtered["dp_dV"])[::-1]]

# Select top 10
top10 = sorted_filtered[:10]

In [80]:
top10

no_GLADE,RAJ2000,DEJ2000,Dist,Bmag,dp_dV
,,,,,1 / Mpc2
uint32,float64,float64,float32,float32,float64
179043,10.432732,46.029392,490.21237,17.24,1.4973702684795808e-14
3406282,10.1923808,45.2480445,482.1057,19.905697,1.0783019827266007e-14
3407467,10.481994199999999,45.981254799999995,360.9565,19.660616,9.131025281980618e-15
3407479,10.2857756,46.0894721,443.26828,18.51903,8.735329455160977e-15
3407499,10.3471844,46.04116679999999,429.48743,19.908466,8.055771979866971e-15
3272839,9.5606882,45.4293756,491.4394,18.74926,7.958657794927129e-15
3406758,10.007826099999999,45.5616666,496.76895,18.890467,7.667408260422347e-15
3406920,9.6782758,45.508192,457.45877,19.700712,7.235365220330726e-15
